# Using mintalib with polars (experimental)


In [1]:
import  numpy as np
import polars as pl
from polars import col

from mintalib.indicators import SMA, ATR, MACD
from mintalib.core import calc_sma, calc_atr, calc_macd

In [2]:
from mintalib.samples import sample_prices

prices = sample_prices()
prices = pl.from_pandas(prices, include_index=True)
prices

# Links
# https://docs.pola.rs/user-guide/expressions/user-defined-python-functions/#combining-multiple-column-values

date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [3]:
pstruct = prices.to_struct("prices")
pstruct


prices
struct[6]
"{1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600}"
"{1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800}"
"{1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000}"
"{1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600}"
"{1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600}"
…
"{2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400}"
"{2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200}"
"{2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800}"


In [4]:
from polars.datatypes import Struct, Float64

macd_type = Struct({'macd': Float64, 'macdsignal': Float64, 'macdhist': Float64})

def my_sma(series, period: int = 20):
    print(f"SMA {type(series).__name__}")
    return calc_sma(series, period)

def my_atr(prices, period: int = 14):
    print(f"ATR {type(prices).__name__}")
    if hasattr(prices, 'dtype') and prices.dtype.__class__.__name__ == 'Struct':
        print("struct ...")
        prices = prices.struct
    return calc_atr(prices, period)

def my_macd(series):
    print(f"MACD {type(series).__name__}")
    result = calc_macd(series, wrap=True)
    if hasattr(result, 'to_struct'):
        return result.to_struct("macd")


df = prices.with_columns(
    col('close').map_batches(my_sma).alias("sma"),
    col('close').map_batches(my_macd).alias("macd"),
    pl.struct(pl.all()).map_batches(my_atr).alias("atr")
)

df

SMA SeriesMACD Series
ATR Series
struct ...



date,open,high,low,close,volume,sma,macd,atr
datetime[ns],f64,f64,f64,f64,i64,f64,struct[3],f64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600,NaN,"{NaN,NaN,NaN}",NaN
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800,NaN,"{NaN,NaN,NaN}",NaN
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000,NaN,"{NaN,NaN,NaN}",NaN
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600,NaN,"{NaN,NaN,NaN}",NaN
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600,NaN,"{NaN,NaN,NaN}",NaN
…,…,…,…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400,227.524,"{1.815958,1.313965,0.501993}",4.516121
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200,228.0785,"{1.941114,1.439395,0.501719}",4.479971
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800,228.2425,"{2.046565,1.560829,0.485736}",4.39783


In [5]:
df.dtypes


[Datetime(time_unit='ns', time_zone=None),
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 Struct({'macd': Float64, 'macdsignal': Float64, 'macdhist': Float64}),
 Float64]

In [6]:
sma = calc_sma(prices['close'], 20, wrap=True)
sma

""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [7]:
SMA(20) @ prices['close']


""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [8]:
SMA(20) @ prices

""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [9]:
MACD() @ prices

macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [10]:
ATR() @ prices

""
f64
NaN
NaN
NaN
NaN
NaN
…
4.516121
4.479971
4.39783


In [11]:
ATR() @ pstruct


""
f64
NaN
NaN
NaN
NaN
NaN
…
4.516121
4.479971
4.39783


In [12]:
prices.select(
    pl.col('close').map_batches(SMA(20))
)

close
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [13]:
df.select([
    pl.struct(["high", "low", "close"])
])



high
struct[3]
"{0.099373,0.098943,0.098943}"
"{0.094211,0.093781,0.093781}"
"{0.087328,0.086898,0.086898}"
"{0.089479,0.089049,0.089049}"
"{0.092061,0.09163,0.09163}"
…
"{237.490005,232.369995,233.850006}"
"{232.119995,229.839996,231.779999}"
"{233.850006,230.520004,232.149994}"


In [17]:
prices.select(
    pl.struct(pl.col("*"))
)



date
struct[6]
"{1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600}"
"{1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800}"
"{1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000}"
"{1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600}"
"{1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600}"
…
"{2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400}"
"{2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200}"
"{2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800}"


In [ ]:
prices.select(
    pl.struct(pl.col("high", "low")).alias("high_low")
)


DuplicateError: multiple fields with name 'high_low' found

In [ ]:


prices.select(
    pl.struct('*').map_batches(ATR())
)


In [ ]:

prices.select(
    pl.struct(["high", "low", "close"]).map_batches(ATR())
)

In [18]:
prices.select(
    pl.col('close').map_batches(my_macd, macd_type)
)


MACD Series


close
struct[3]
"{NaN,NaN,NaN}"
"{NaN,NaN,NaN}"
"{NaN,NaN,NaN}"
"{NaN,NaN,NaN}"
"{NaN,NaN,NaN}"
…
"{1.815958,1.313965,0.501993}"
"{1.941114,1.439395,0.501719}"
"{2.046565,1.560829,0.485736}"


In [19]:
prices.select(
    pl.col('close').map_batches(my_macd, macd_type).struct.unnest()
)

MACD SeriesMACD Series

MACD Series


macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [20]:
prices.select(
    pl.col('close').map_batches(my_macd, macd_type).struct[0]
)

MACD Series


macd
f64
NaN
NaN
NaN
NaN
NaN
…
1.815958
1.941114
2.046565


In [21]:
prices.select(
    pl.col('close').map_batches(my_macd, macd_type).struct.field("macd")
)


MACD Series


macd
f64
NaN
NaN
NaN
NaN
NaN
…
1.815958
1.941114
2.046565
